# Notebook que permite resolver el problema del Knapsack con valores asociados

En este Notebook se implementa el algoritmo que permite resolver el problema de la mochila, la idea es encontrar la configuración de objetos, cada uno asociado con un valor y un peso, que maximiza el valor total de los objetos seleccionados pero sin superar la capacidad C de la mochila.

Versiones:
- v1. Aprovechamiento de la sparsity del problema e implementación en GPU, sirve para resolver problemas donde cada objeto puede ser seleccionado i veces.
- v2. Aprovechamiento de la sparsity de los tensores generados teniendo en cuenta que solamente se forman dos diagonales no nulas, solo permite resolver el 0-1 knapsack
    

    


In [2]:
#Librerías
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from time import time
from scipy.sparse import diags


---
# Funciones de la tensor network

## Definimos los nodos de la tensor network:

In [ ]:

def tensor_initial_generator(peso: int, valor: int, n_elementos: int, capacidad: int,previous_weight_solution: int) -> csr_matrix :
    """
    Genera el primer tensor en formato sparse de la cadena.

    Parameters:
    - peso (int): Peso del primer elemento.
    - valor (int): Valor del primer elemento.
    - n_elementos (int): numero de veces que se repite el primer elemento, para el caso del 0-1 knapsack siempre vale uno.
    - capacidad (int): capacidad limite de la mochila.
    - previous_weight_solution (int): peso que se pasa desde arriba.


    Returns:
    - csr_matrix: representación sparse del primer tensor.

    """
    # Calculamos las dimensiones del tensor inicial
    tamaño_1 = min((capacidad - previous_weight_solution) // peso, n_elementos) + 1
    tamaño_2 = min(capacidad, (tamaño_1 - 1) * peso + previous_weight_solution) + 1
    tensor = lil_matrix((tamaño_1, tamaño_2))
    tensor[0,previous_weight_solution] = 1
    tensor[tamaño_1-1,tamaño_2-1] = valor
    tensor_shape = (tamaño_1,tamaño_2)
    return tensor, tensor_shape

def tensor_intermediate_generator(peso: int, valor: int, n_elementos: int, capacidad: int, previous_weight: int) -> csr_matrix:
    
    tamaño_1 = previous_weight
    tamaño_2 = min(capacidad, n_elementos * peso + previous_weight - 1) + 1
    '''
    diagonals = [1]
    for k in range(1,n_elementos+1):
        diagonals.append(valor**k)
    positions = [0]
    for k in range(1,n_elementos+1):
        positions.append(peso*k)
    tensor = diags(diagonals, positions, shape=(tamaño_1,tamaño_2))

    non_zero_data = tensor.data[tensor.data != 0]
    unique_elements, counts = np.unique(non_zero_data, return_counts=True)
    
    # Asignar los valores a una lista y las repeticiones a otra
    
    repetitions = counts.tolist()
    '''
    tensor_shape = (tamaño_1,tamaño_2)
    c2=min(tamaño_2-peso,tamaño_1)
    repetitions=[tamaño_1,c2]
    
    return tensor_shape,  repetitions

def tensor_final_generator(peso:int, valor:int,n_elementos: int, capacidad: int, previous_weight:int)->np.array:
    tamaño_1 = previous_weight
    tensor = np.zeros((tamaño_1))
    for i in range(tamaño_1):
        n_elementos_posibles = min((capacidad -i)//peso,n_elementos)+1
        for j in range(n_elementos_posibles):
            tensor[i] += valor**j
    #tensor = np.ones((tamaño_1))
    return tensor

In [ ]:
def tensor_generator(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int):
    
    lista_de_tensores = []
    repeticiones = []
    n_clases = len(pesos)

    tensor,tensor_shape = tensor_initial_generator(pesos[0],valores[0],n_elementos[0],capacidad,0)
    repeticiones.append([1,1])
    #lista_de_tensores.append(tensor)
    

    for n in range(1, n_clases-1):
        tensor_shape,repetitions = tensor_intermediate_generator(pesos[n], valores[n], n_elementos[n], capacidad, tensor_shape[1])
        #lista_de_tensores.append(tensor)
        repeticiones.append(repetitions)


    tensor = tensor_final_generator(pesos[-1], valores[-1], n_elementos[-1], capacidad, tensor_shape[1])

    #lista_de_tensores.append(tensor)
 
    return repeticiones,tensor

### Función de contracción de los tensores

In [5]:
def multiplication(v2,c1,c2,peso,vector):

    # vector2=np.zeros(c1)
    # vector3=vector.copy()[:c1]
    # vector2[:c2]=v2*np.pad(vector[peso:c2+peso], (0, len(vector2[:c2])-len(vector[peso:c2+peso])))
    # vector2=vector2+vector3
    # return vector2
    result = vector[:c1]
    diag_2 = np.zeros(c1)
    diag_2[:c2] = vector[peso:peso+c2]
    diag_2*=v2

    return result+diag_2


In [ ]:
def tensor_contraction(values, pesos,repetitions,vector):

    n_tensores = len(values)
    tensores_intermedios = []
    tensores_intermedios.append(vector)
    current_tensor = vector
    for j in range(n_tensores - 2, 0, -1):

        c1=repetitions[j][0]
        c2=repetitions[j][1]

        current_tensor = multiplication(values[j],c1,c2,pesos[j],current_tensor)

        tensores_intermedios.append(current_tensor)
        if np.max(current_tensor)>1e200:
            current_tensor*=1e-15


    current_tensor = np.array([current_tensor[0],current_tensor[-1]])
    tensores_intermedios.append(current_tensor)
    tensores_intermedios.reverse()
    return current_tensor, tensores_intermedios

### Función general

In [7]:

def solver(
    pesos: np.array, 
    valores: np.array, 
    n_elementos: np.array, 
    capacidad: int, 
    tao: float
) -> tuple:

   
    valores_scaled = np.exp(valores * tao)
   
    n_clases = len(pesos)
    solution = np.zeros(n_clases)
    in1=time()
    repetitions, vector = tensor_generator(pesos, valores_scaled, n_elementos, capacidad)

    fn1=time()
    print("tiempo creacion",fn1-in1)
    #Compute output vector and intermediate tensors through contraction
    in2=time()
    vector_salida, tensores_intermedios = tensor_contraction(valores_scaled, pesos,repetitions,vector)
    fn2=time()
    print("tiempo comprsion",fn2-in2)

    #Determine the maximum value and initialize the solution vector
    max_value = np.max(tensores_intermedios[0])
 
    solution[0] = np.argmax(abs(vector_salida))

    in3=time()
    # Iterate over classes to build the solution progressively
    for n in range(1, n_clases - 1):

        cumulative_weight = int(np.dot(solution[:n], pesos[:n]))
    
        new_initial_tensor,tensor_shape = tensor_initial_generator(
            pesos[n], valores_scaled[n], n_elementos[n], capacidad, cumulative_weight
        )
        if n == n_clases - 2:
            tensores_intermedios[n + 1] = tensores_intermedios[n + 1].T  # Transpose for final calculation

        solution[n] = np.argmax(abs(new_initial_tensor @ tensores_intermedios[n + 1][:tensor_shape[1]]))
   

    fn3=time()
    print("tiempo resto",fn3-in3)
    cumulative_weight = np.dot(solution[:-1], pesos[:-1])
    solution[-1] = min((capacidad - cumulative_weight) // pesos[-1], n_elementos[-1])


    total_weight = np.dot(solution, pesos)
    total_value = np.dot(solution, valores)
    
    print("El peso total es:", total_weight)
    print("El valor total es:", total_value)

    return total_weight, total_value, max_value

### Funciones auxiliares para la realización de las pruebas

In [8]:
def mochila_greedy(pesos, valores, capacidad, n_elementos):
    n_clases = len(pesos)
    
    # Calculamos la relación valor/peso para cada clase
    valor_peso = valores / pesos
    
    # Ordenamos los elementos por su valor/peso de mayor a menor
    indices_ordenados = np.argsort(valor_peso)[::-1]  # Orden descendente
    
    # Inicializamos variables
    peso_actual = 0
    valor_actual = 0
    solucion = np.zeros(n_clases, dtype=int)
    
    # Recorremos los objetos en orden greedy
    for i in indices_ordenados:
        # Tomamos la mayor cantidad posible de este objeto sin exceder la capacidad
        cantidad = min(n_elementos[i], (capacidad - peso_actual) // pesos[i])
        solucion[i] = cantidad
        peso_actual += cantidad * pesos[i]
        valor_actual += cantidad * valores[i]
        
        # Si llenamos la capacidad, salimos del bucle
        if peso_actual >= capacidad:
            break
    
    return solucion, valor_actual


In [9]:
def leer_archivo_knapsack(ruta_archivo):
    with open(ruta_archivo, 'r') as archivo:
        lineas = archivo.readlines()
    
    # Limpiar líneas en blanco
    lineas = [linea.strip() for linea in lineas if linea.strip()]
    
    # Número de clases (primera línea)
    clases = int(lineas[0].strip())
    
    # Listas para almacenar los pesos y valores
    pesos = []
    valores = []
    
    # Leer ítems (líneas desde la 2 hasta la penúltima)
    for linea in lineas[1:-1]:  # Excluimos la primera y la última línea
        datos = linea.split()
        if len(datos) != 3:
            print(f"Advertencia: Línea no tiene 3 valores: {linea}")
            continue  # O puedes manejar este caso según sea necesario
        id_, peso, valor = datos
        pesos.append(int(peso))
        valores.append(float(valor))
    
    # Capacidad de la mochila (última línea)
    capacidad = int(lineas[-1].strip())
    
    return clases, pesos, valores, capacidad

# Ruta del archivo
ruta = '/home/sergio/simulador/Simulador_Cuantica/quantum-sim/TensorNetwork/knapsack/knapsackProblemInstances/problemInstances/n_400_c_1000000_g_2_f_0.1_eps_0_s_100/test.in'

# Leer archivo
clases, pesos, valores, capacidad = leer_archivo_knapsack(ruta)

# Mostrar los datos leídos
print(f"Número de clases: {clases}")
print(f"Pesos de los ítems: {pesos}")  # Mostrar los primeros 5 pesos
print(f"Valores de los ítems: {valores}")  # Mostrar los primeros 5 valores
print(f"Capacidad de la mochila: {capacidad}")

Número de clases: 400
Pesos de los ítems: [500014, 500090, 500086, 500086, 500048, 500054, 500002, 500036, 500075, 500028, 500011, 500075, 500099, 500030, 500074, 500090, 500097, 500058, 500097, 500013, 500003, 500087, 500058, 500035, 500048, 500034, 500070, 500043, 500007, 500068, 500027, 500067, 500024, 500048, 500006, 500091, 500066, 500052, 500013, 500011, 500038, 500040, 500070, 500028, 500002, 500093, 500022, 500015, 500010, 500050, 500080, 500029, 500049, 500087, 500006, 500009, 500013, 500005, 500018, 500082, 500073, 500036, 500067, 500089, 500047, 500016, 500070, 500013, 500026, 500074, 500063, 500058, 500075, 500044, 500052, 500057, 500049, 500001, 500087, 500034, 500063, 500084, 500069, 500093, 500088, 500010, 500054, 500016, 500089, 500011, 500026, 500010, 500039, 500014, 500051, 500087, 500096, 500083, 500094, 500079, 500002, 500061, 500072, 500084, 500081, 500005, 500049, 500044, 500005, 500056, 500019, 500065, 500004, 500006, 500050, 500056, 500030, 500081, 500050, 50005

In [17]:

clases = 20000
capacidad = 18756
np.random.seed(1)
pesos = np.random.randint(1,10,clases)



valores = np.random.rand(clases)

pesos = np.array(pesos)

valores = np.array(valores)

n_elementos = [1]*clases





tao = 10

inicio = time()
b=solver(pesos,valores, n_elementos, capacidad, tao)
fin = time()
print("tiempo de ejecución", fin-inicio)
mejor_solucion, mejor_valor = mochila_greedy(pesos, valores, capacidad, n_elementos)

#print("Mejor solución encontrada (greedy):", mejor_solucion)
print("Valor total de la mejor solución (greedy):", mejor_valor)
print("Peso total de la mejor solución (greedy):", np.sum(mejor_solucion * pesos))


tiempo creacion 0.05754733085632324
1
tiempo comprsion 1.0547184944152832
tiempo resto 1.1633753776550293
El peso total es: 18754.0
El valor total es: 3106.1414608977284
tiempo de ejecución 2.324859619140625
Valor total de la mejor solución (greedy): 4741.492699011403
Peso total de la mejor solución (greedy): 18756


In [11]:
import matplotlib.pyplot as plt
import random
from time import time
def guardar_experimento(nombre_archivo, descripcion_experimento, precision,tao, max_values,timet,capacity):
    with open(nombre_archivo, 'a') as file:
        file.write(f"Descripción del experimento: {descripcion_experimento}\n")
        file.write(f"Precisión: {precision}\n")
        file.write(f"Valores tao: {tao}\n")
        file.write(f"Valores máximos: {max_values}\n")
        file.write(f"tiempos: {timet}\n")
        file.write(f"capacidad: {capacity}\n")
        file.write("="*40 + "\n")
        
def tao_experimento(clases,capacidad,pesos,valores,n_elementos,tao,lambda1,experimento):
   precision=[]
   max_values=[]
   time = []
   for t in tao:
      inicio = time()
      p1,v1,max=solver(pesos,valores,n_elementos,capacidad,t,lambda1)
      fin = time()
      tiempo = fin-inicio
      p2,v2 = mochila_greedy(pesos,valores,capacidad,n_elementos)
      time.append(tiempo)
      precision.append(v1/v2)
      max_values.append(max)
   random_color = (random.random(), random.random(), random.random())
   plt.plot(tao, precision, marker='o', color=random_color, linestyle='-')  # 'o' para puntos, 'b' para color azul, '-' para línea sólida
   plt.axhline(y=1, color='gray', linestyle='--', linewidth=0.8)
   # Añadir etiquetas y título
   plt.xlabel(r"$\tau$")
   plt.ylabel("precision")
   plt.title("Gráfico de línea con dos listas")
   #plt.savefig(f"grafico_precision_{experimento}.png", format="png", dpi=300)
   nombre_archivo = f"resultados_experimentos.txt"
   descripcion_experimento = f"Experimento {experimento}, clases={clases} con min_tao={tao[0]} ,max_tao={tao[-1]},capacidad={int(0.75*(clases)*n_elementos[0])},n_elementos={n_elementos}, tiempo ={time}"
   guardar_experimento(nombre_archivo, descripcion_experimento, precision,tao, max_values,capacidad)
   return precision,max_values

In [12]:
'''
from time import time
np.random.seed(1)

capacidad = int(0.75*(clases)*n_elementos[0])
tao = np.linspace(0,8,12)
timet = []
clase = []
capacity = []
lambda1 = 0
exp=1
tao = 5
lastexp=15
for exp in range(1,lastexp):
    clases = 10000
    
    pesos = np.random.randint(1,10,clases)
    capacidad = int(exp*1*(clases)*n_elementos[0])
    capacity.append(capacidad)
    valores = np.random.rand(clases)
    n_elementos = [1]*clases
    inicio=time()
    precision,max_values,max=solver(pesos,valores,n_elementos,capacidad,tao)
    fin = time()
    tiempo = fin-inicio
    timet.append(tiempo)

random_color = (random.random(), random.random(), random.random())
plt.plot(capacity, timet, marker='o', color=random_color, linestyle='-')  # 'o' para puntos, 'b' para color azul, '-' para línea sólida
plt.axhline(y=1, color='gray', linestyle='--', linewidth=0.8)
# Añadir etiquetas y título
plt.xlabel("capacidad en porciento")
plt.ylabel("tiempo")
plt.title("Gráfico de línea con dos listas")
#plt.savefig(f"grafico_precision_{experimento}.png", format="png", dpi=300)
nombre_archivo = f"resultados_experimentos.txt"
descripcion_experimento = f"Experimento tiempo en función de clase, clases={clases} con tao={tao},capacidad={capacidad},n_elementos={n_elementos}, tiempo ={timet}"
guardar_experimento(nombre_archivo, descripcion_experimento, precision,tao, max_values, timet,capacity)
'''

'\nfrom time import time\nnp.random.seed(1)\n\ncapacidad = int(0.75*(clases)*n_elementos[0])\ntao = np.linspace(0,8,12)\ntimet = []\nclase = []\ncapacity = []\nlambda1 = 0\nexp=1\ntao = 5\nlastexp=15\nfor exp in range(1,lastexp):\n    clases = 10000\n    \n    pesos = np.random.randint(1,10,clases)\n    capacidad = int(exp*1*(clases)*n_elementos[0])\n    capacity.append(capacidad)\n    valores = np.random.rand(clases)\n    n_elementos = [1]*clases\n    inicio=time()\n    precision,max_values,max=solver(pesos,valores,n_elementos,capacidad,tao)\n    fin = time()\n    tiempo = fin-inicio\n    timet.append(tiempo)\n\nrandom_color = (random.random(), random.random(), random.random())\nplt.plot(capacity, timet, marker=\'o\', color=random_color, linestyle=\'-\')  # \'o\' para puntos, \'b\' para color azul, \'-\' para línea sólida\nplt.axhline(y=1, color=\'gray\', linestyle=\'--\', linewidth=0.8)\n# Añadir etiquetas y título\nplt.xlabel("capacidad en porciento")\nplt.ylabel("tiempo")\nplt.titl